In [1]:
! pip install confluent_kafka

     |████████████████████████████████| 3.1 MB 21.2 MB/s eta 0:00:01


In [2]:
import json
import pprint

import confluent_kafka as ck
import requests
import psycopg2
import pandas as pd

## Kafka settings

In [3]:
"""
Create kafka consumer / producer instances
"""

consumer = ck.Consumer(
    {
        "bootstrap.servers": "kafka:9092",
        "group.id": "teste",
        "auto.offset.reset": "earliest",
    }
)

producer = ck.Producer(
    {
        "bootstrap.servers": "kafka:9092",
    }
)

In [4]:
# topics = consumer.list_topics()
# pprint.pprint(topics.topics)

In [28]:
"""
Subscribe car price table
- detect of table changes
- we can use poll to get the events created for given timeout interval (1 seconeds for this example)
"""

consumer.subscribe(["car_database.public.tbl_car_price"])

Create a new record in car database.

```sql
INSERT INTO
    tbl_car_price
(
    MODEL,
    YEAR,
    TRANSMISSION,
    MILEAGE,
    FUEL_TYPE,
    TAX,
    MPG,
    ENGINE_SIZE
)
VALUES
(
    'C-MAX',
    2016,
    'Manual',
    5060,
    'Diesel',
    165,
    45.2,
    1.2
);
```

In [41]:
msg = consumer.poll(1.0)

In [42]:
json_msg = json.loads(msg.value().decode("utf-8"))
pprint.pprint(json_msg)

{'payload': {'engine_size': 1.2,
             'fuel_type': 'Diesel',
             'id': 5,
             'mileage': 5060,
             'model': 'C-MAX',
             'mpg': 45.2,
             'price': None,
             'suggested_price': None,
             'tax': 165,
             'transmission': 'Manual',
             'year': 2017},
 'schema': {'fields': [{'default': 0,
                        'field': 'id',
                        'optional': False,
                        'type': 'int32'},
                       {'field': 'model', 'optional': True, 'type': 'string'},
                       {'field': 'year', 'optional': True, 'type': 'int32'},
                       {'field': 'price', 'optional': True, 'type': 'int32'},
                       {'field': 'transmission',
                        'optional': True,
                        'type': 'string'},
                       {'field': 'mileage', 'optional': True, 'type': 'int32'},
                       {'field': 'fuel_type',
        

## Bentoml API request
Before post data to the API server, don't forget to run `bento serve` in docker container with:
```bash
$ bentoml serve --host price_prediction_service --host 0.0.0.0 --port 3000 --production
```

In [43]:
FIELDS = {
    "model": "model",
    "year": "year",
    "transmission": "transmission",
    "mileage": "mileage",
    "fuel_type": "fuelType",
    "tax": "tax",
    "mpg": "mpg",
    "engine_size": "engineSize",
}

In [44]:
ml_inputs = {
    api_col: [json_msg["payload"][db_col]]
    for db_col, api_col in FIELDS.items()
}
pprint.pprint(ml_inputs)

{'engineSize': [1.2],
 'fuelType': ['Diesel'],
 'mileage': [5060],
 'model': ['C-MAX'],
 'mpg': [45.2],
 'tax': [165],
 'transmission': ['Manual'],
 'year': [2017]}


In [45]:
"""
Request the BentoML deoloyed server to predict the price
"""

endpoint = "http://0.0.0.0:3000/predict"
response = requests.post(endpoint, json=ml_inputs)

print(f"status: {response.status_code}")
print(f"output: {json.loads(response.text)}")

status: 200
output: [10200.7275390625]


In [46]:
"""
Append prediction result to the response we got from the consumer.
"""

json_msg["payload"]["suggested_price"] = float(json.loads(response.text)[0])
pprint.pprint(json_msg["payload"])

{'engine_size': 1.2,
 'fuel_type': 'Diesel',
 'id': 5,
 'mileage': 5060,
 'model': 'C-MAX',
 'mpg': 45.2,
 'price': None,
 'suggested_price': 10200.7275390625,
 'tax': 165,
 'transmission': 'Manual',
 'year': 2017}


## Postgres DB status

In [47]:
"""
Check out current db status
"""

conn = psycopg2.connect(
    host="postgres",
    port="5432",
    user="postgres",
    password="postgres",
    database="database",
)

In [48]:
pd.read_sql_query("select * from public.tbl_car_price", conn)

/opt/conda/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,model,year,price,transmission,mileage,fuel_type,tax,mpg,engine_size,suggested_price
0,1,Fiesta,2017,None,Automatic,13000,Petrol,140,43.2,1.3,None
1,2,B-MAX,2020,None,Manual,4000,Diesel,100,57.4,1.5,None
2,3,C-MAX,2016,None,Manual,5060,Diesel,165,45.2,1.2,None
3,4,C-MAX,2016,None,Manual,5060,Diesel,165,45.2,1.2,None
4,5,C-MAX,2017,None,Manual,5060,Diesel,165,45.2,1.2,None
5,6,B-MAX,2020,None,Manual,4000,Diesel,100,57.4,2.0,None
6,7,C-MAX,2016,None,Manual,5060,Diesel,165,45.2,1.2,None
7,8,C-MAX,2016,None,Manual,5060,Diesel,165,45.2,1.2,None
8,9,C-MAX,2016,None,Manual,5060,Diesel,165,45.2,1.3,None


In [49]:
"""
Send the message to the topic with the predicted price
"""

producer.produce(
    "car_data_predicted",
    key=msg.key().decode("utf-8"),
    value=json.dumps(json_msg)
)
producer.flush(1)

0

In [50]:
"""
Now, the topic "car_data_predicted" is updated as you can see below
"""

pd.read_sql_query("select * from public.car_data_predicted", conn)

/opt/conda/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,transmission,year,mpg,price,suggested_price,model,tax,engine_size,id,fuel_type,mileage
0,Manual,2016,45.2,None,10369,C-MAX,165,1.2,3,Diesel,5060
1,Manual,2016,45.2,None,10369,C-MAX,165,1.2,4,Diesel,5060
2,Manual,2017,45.2,None,10200,C-MAX,165,1.2,5,Diesel,5060


In [60]:
type(msg.value())

bytes

In [62]:
isinstance(b'dd', bytes)

True

In [66]:
msg.value().decode("utf-8")

'{"schema":{"type":"struct","fields":[{"type":"int32","optional":false,"default":0,"field":"id"},{"type":"string","optional":true,"field":"model"},{"type":"int32","optional":true,"field":"year"},{"type":"int32","optional":true,"field":"price"},{"type":"string","optional":true,"field":"transmission"},{"type":"int32","optional":true,"field":"mileage"},{"type":"string","optional":true,"field":"fuel_type"},{"type":"int32","optional":true,"field":"tax"},{"type":"double","optional":true,"field":"mpg"},{"type":"double","optional":true,"field":"engine_size"},{"type":"int32","optional":true,"field":"suggested_price"}],"optional":false,"name":"car_database.public.tbl_car_price.Value"},"payload":{"id":5,"model":"C-MAX","year":2017,"price":null,"transmission":"Manual","mileage":5060,"fuel_type":"Diesel","tax":165,"mpg":45.2,"engine_size":1.2,"suggested_price":null}}'

In [67]:
response.text

'[10200.7275390625]'